In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import random
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                             stride=stride, padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dilation=1, downsample=None):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, 
                              stride=1, padding=dilation, dilation=dilation, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, output_stride=16):
        super(ResNet, self).__init__()
        
        if output_stride == 16:
            strides = [1, 2, 2, 1]
            dilations = [1, 1, 1, 2]
        elif output_stride == 8:
            strides = [1, 2, 1, 1]
            dilations = [1, 1, 2, 4]
        else:
            raise NotImplementedError
        
        self.inplanes = 64
        
        # Initial layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Residual blocks
        self.layer1 = self._make_layer(block, 64, layers[0], stride=strides[0], dilation=dilations[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=strides[1], dilation=dilations[1])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=strides[2], dilation=dilations[2])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=strides[3], dilation=dilations[3])
        
    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
            
        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation, downsample))
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        low_level_feat = x  # Output stride: 4
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        return x, low_level_feat

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, output_stride=16):
        super(ASPP, self).__init__()
        
        if output_stride == 16:
            dilations = [1, 6, 12, 18]
        elif output_stride == 8:
            dilations = [1, 12, 24, 36]
        else:
            raise NotImplementedError
            
        self.aspp1 = ConvBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=dilations[0])
        self.aspp2 = ConvBlock(in_channels, out_channels, kernel_size=3, stride=1, 
                              padding=dilations[1], dilation=dilations[1])
        self.aspp3 = ConvBlock(in_channels, out_channels, kernel_size=3, stride=1, 
                              padding=dilations[2], dilation=dilations[2])
        self.aspp4 = ConvBlock(in_channels, out_channels, kernel_size=3, stride=1, 
                              padding=dilations[3], dilation=dilations[3])
        
        self.global_avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            ConvBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        )
        
        self.conv1 = ConvBlock(out_channels * 5, out_channels, kernel_size=1, stride=1, padding=0)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)
        
        x = torch.cat((x1, x2, x3, x4, x5), dim=1)
        
        x = self.conv1(x)
        x = self.dropout(x)
        
        return x

class DeepLabv3PlusDecoder(nn.Module):
    def __init__(self, low_level_channels, aspp_channels, num_classes):
        super(DeepLabv3PlusDecoder, self).__init__()
        
        # Low-level feature processing
        self.low_level_conv = nn.Sequential(
            nn.Conv2d(low_level_channels, 48, kernel_size=1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )
        
        # Fusion and final prediction
        self.fusion = nn.Sequential(
            nn.Conv2d(aspp_channels + 48, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Conv2d(256, num_classes, kernel_size=1)
        )
        
    def forward(self, aspp_features, low_level_features):
        # Process low-level features
        low_level_features = self.low_level_conv(low_level_features)
        
        # Upsample ASPP features to the same size as low-level features
        aspp_features = F.interpolate(
            aspp_features, 
            size=low_level_features.shape[2:], 
            mode='bilinear', 
            align_corners=True
        )
        
        # Concatenate and process
        concat_features = torch.cat([aspp_features, low_level_features], dim=1)
        output = self.fusion(concat_features)
        
        return output

class DeepLabv3Plus(nn.Module):
    def __init__(self, n_channels=3, n_classes=1, output_stride=16, deep_supervision=False):
        super(DeepLabv3Plus, self).__init__()
        self.deep_supervision = deep_supervision
        
        # Define backbone (ResNet)
        self.backbone = ResNet(ResBlock, [3, 4, 6, 3], output_stride=output_stride)
        
        # ASPP module
        self.aspp = ASPP(512, 256, output_stride=output_stride)
        
        # Decoder (DeepLabv3+ specific)
        self.decoder = DeepLabv3PlusDecoder(low_level_channels=64, aspp_channels=256, num_classes=n_classes)
        
        # Deep supervision (if needed)
        if self.deep_supervision:
            self.aux_classifier = nn.Sequential(
                ConvBlock(256, 256, kernel_size=3, stride=1, padding=1),
                nn.Dropout(0.1),
                nn.Conv2d(256, n_classes, kernel_size=1, stride=1)
            )
    
    def forward(self, x):
        input_size = x.size()
        
        # Backbone
        encoder_output, low_level_feat = self.backbone(x)
        
        # ASPP
        aspp_output = self.aspp(encoder_output)
        
        # DeepLabv3+ Decoder
        decoder_output = self.decoder(aspp_output, low_level_feat)
        
        # Upsample to input resolution
        output = F.interpolate(
            decoder_output,
            size=input_size[2:],
            mode='bilinear',
            align_corners=True
        )
        
        if self.deep_supervision and self.training:
            # Auxiliary loss
            aux = self.aux_classifier(aspp_output)
            aux = F.interpolate(aux, size=input_size[2:], mode='bilinear', align_corners=True)
            return [output, aux]
        
        return output



In [9]:
import os
import random
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchvision import transforms

# ---------------- Dataset Class ----------------
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, image_filenames=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        if image_filenames is None:
            self.images = [img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')]
        else:
            self.images = image_filenames

        self.image_paths, self.mask_paths = [], []

        for img_name in self.images:
            img_path = os.path.join(self.image_dir, img_name)
            base_name = os.path.splitext(img_name)[0]

            mask_path = None
            for suffix in ['_annotation', '_Annotation']:
                for ext in ['.png', '.jpg', '.jpeg', '.tif', '.bmp']:
                    potential_path = os.path.join(self.mask_dir, base_name + suffix + ext)
                    if os.path.exists(potential_path):
                        mask_path = potential_path
                        break
                if mask_path:
                    break

            if os.path.exists(img_path) and mask_path:
                self.image_paths.append(img_path)
                self.mask_paths.append(mask_path)
            else:
                print(f"Warning: Skipping {img_name} - corresponding mask not found")

        print(f"Found {len(self.image_paths)} valid image-mask pairs out of {len(self.images)} images")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        mask = Image.open(self.mask_paths[idx]).convert('L')

        image = np.array(image)
        mask = np.array(mask)

        # Optional: ensure size match before albumentations (helps prevent mismatch bugs)
        if image.shape[:2] != mask.shape[:2]:
            min_height = min(image.shape[0], mask.shape[0])
            min_width = min(image.shape[1], mask.shape[1])
            image = image[:min_height, :min_width]
            mask = mask[:min_height, :min_width]

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.unsqueeze(0).float() / 255.


# ---------------- Transforms ----------------
def get_transforms(train=True, target_size=(256, 256)):
    try:
        from albumentations import Compose, HorizontalFlip, VerticalFlip, Normalize, RandomRotate90, Resize
        from albumentations.pytorch import ToTensorV2

        if train:
            return Compose([
                Resize(*target_size),
                HorizontalFlip(p=0.5),
                VerticalFlip(p=0.5),
                RandomRotate90(p=0.5),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            return Compose([
                Resize(*target_size),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    except ImportError:
        raise ImportError("Albumentations is not installed. Please install it for transformations.")


# ---------------- Train/Val Dataset Split ----------------
def create_train_val_datasets(image_dir, mask_dir, val_split=0.2, random_state=42, target_size=(512, 512)):
    image_filenames = [img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')]

    train_filenames, val_filenames = train_test_split(image_filenames, test_size=val_split, random_state=random_state)

    train_dataset = SegmentationDataset(
        image_dir, mask_dir,
        transform=get_transforms(train=True, target_size=target_size),
        image_filenames=train_filenames
    )

    val_dataset = SegmentationDataset(
        image_dir, mask_dir,
        transform=get_transforms(train=False, target_size=target_size),
        image_filenames=val_filenames
    )

    return train_dataset, val_dataset


# ---------------- Loss Function ----------------
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, inputs, targets, smooth=1):
        inputs_sigmoid = torch.sigmoid(inputs)
        bce = self.bce(inputs, targets)
        intersection = (inputs_sigmoid * targets).sum()
        dice = (2.*intersection + smooth)/(inputs_sigmoid.sum() + targets.sum() + smooth)
        return 1 - dice + bce


# ---------------- Metrics ----------------
def dice_coeff(preds, targets, smooth=1.):
    preds = (torch.sigmoid(preds) > 0.5).float().view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    return ((2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)).item()

def iou_score(preds, targets, smooth=1.):
    preds = (torch.sigmoid(preds) > 0.5).float().view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def accuracy_score(preds, targets):
    preds = (torch.sigmoid(preds) > 0.5).float()
    return (preds == targets).float().mean().item()

def precision_recall_f1(preds, targets, smooth=1e-6):
    preds = (torch.sigmoid(preds) > 0.5).float()
    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()
    precision = tp / (tp + fp + smooth)
    recall = tp / (tp + fn + smooth)
    f1 = 2 * precision * recall / (precision + recall + smooth)
    return precision, recall, f1


# ---------------- Training ----------------
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for images, masks in tqdm(dataloader):
        images, masks = images.to(device), masks.to(device)
        preds = model(images)
        loss = criterion(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)


# ---------------- Validation ----------------
def validate(model, dataloader, criterion, device):
    model.eval()
    val_loss = dice_total = iou_total = acc_total = precision_total = recall_total = f1_total = 0
    with torch.no_grad():
        for images, masks in tqdm(dataloader):
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            masks = masks.squeeze(1) if masks.ndim > 4 else masks

            loss = criterion(preds, masks)
            val_loss += loss.item()
            dice_total += dice_coeff(preds, masks)
            iou_total += iou_score(preds, masks)
            acc_total += accuracy_score(preds, masks)
            p, r, f1 = precision_recall_f1(preds, masks)
            precision_total += p
            recall_total += r
            f1_total += f1

    n = len(dataloader)
    return {
        'val_loss': val_loss / n,
        'dice': dice_total / n,
        'iou': iou_total / n,
        'accuracy': acc_total / n,
        'precision': precision_total / n,
        'recall': recall_total / n,
        'f1': f1_total / n
    }


# ---------------- Save Predictions ----------------
def save_predictions(model, dataloader, save_dir, device):
    model.eval()
    os.makedirs(save_dir, exist_ok=True)
    with torch.no_grad():
        for idx, (images, _) in enumerate(dataloader):
            images = images.to(device)
            preds = model(images)
            preds = (torch.sigmoid(preds) > 0.5).float()

            for i in range(images.size(0)):
                pred_np = preds[i].squeeze().cpu().numpy() * 255
                pred_img = Image.fromarray(pred_np.astype(np.uint8))
                pred_img.save(os.path.join(save_dir, f"pred_{idx*images.size(0)+i}.png"))


# ---------------- Train Model with Enhanced Weight Saving ----------------
def train_model(model, train_loader, val_loader, criterion, optimizer, device, 
                epochs=10, save_dir="weights", save_best=True, save_last=True, save_interval=5):
    """
    Train the model with enhanced weight saving options:
    - save_best: Save the best model based on validation dice coefficient
    - save_last: Save the model after the last epoch
    - save_interval: Save the model every 'save_interval' epochs
    """
    os.makedirs(save_dir, exist_ok=True)
    best_dice = 0
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        metrics = validate(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {metrics['val_loss']:.4f}, Dice: {metrics['dice']:.4f}, IoU: {metrics['iou']:.4f}, " +
              f"Acc: {metrics['accuracy']:.4f}, Prec: {metrics['precision']:.4f}, " +
              f"Recall: {metrics['recall']:.4f}, F1: {metrics['f1']:.4f}")

        # Save model at regular intervals
        if save_interval > 0 and (epoch + 1) % save_interval == 0:
            interval_save_path = os.path.join(save_dir, f"unetpp_epoch_{epoch+1}.pth")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_metrics': metrics
            }, interval_save_path)
            print(f"✅ Saved checkpoint at epoch {epoch+1}")

        # Save best model based on validation dice
        if save_best and metrics['dice'] > best_dice:
            best_dice = metrics['dice']
            best_save_path = os.path.join(save_dir, "best_unetpp.pth")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_dice': best_dice,
                'val_metrics': metrics
            }, best_save_path)
            print(f"✅ Saved best model with Dice: {best_dice:.4f}")
    
    # Save model after the last epoch
    if save_last:
        last_save_path = os.path.join(save_dir, "last_unetpp.pth")
        torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_metrics': metrics
        }, last_save_path)
        print(f"✅ Saved model after last epoch")
    
    return best_dice

# ---------------- Load Model Weights ----------------
def load_model(model, weights_path, device):
    """
    Load model weights from a checkpoint
    """
    checkpoint = torch.load(weights_path, map_location=device)
    
    # Check if the checkpoint contains the full dictionary or just the state_dict
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Loaded model from epoch {checkpoint['epoch']} with Dice score: {checkpoint.get('best_dice', 'N/A')}")
    else:
        model.load_state_dict(checkpoint)
        print(f"✅ Loaded model weights")
    
    return model

# ---------------- Main ----------------
if __name__ == '__main__':
    # Create directories for weights and predictions
    os.makedirs("weights", exist_ok=True)
    os.makedirs("predictions", exist_ok=True)
    
    # Set paths to your single image and mask folders
    image_dir = './data/images'
    mask_dir = './data/masks'

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize model
    model = DeepLabv3Plus(n_channels=3, n_classes=1, deep_supervision=False).to(device)
    criterion = DiceBCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    # Define image target size for resizing
    target_size = (256, 256)  # Common size for segmentation tasks
    
    # Split the dataset and create data loaders with resized images
    train_dataset, val_dataset = create_train_val_datasets(
        image_dir, 
        mask_dir, 
        val_split=0.2,  # 20% for validation
        random_state=42,  # For reproducibility
        target_size=target_size  # Target size for resizing
    )

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)
    
    # Train model with enhanced weight saving
    best_dice = train_model(
        model, 
        train_loader, 
        val_loader, 
        criterion, 
        optimizer, 
        device, 
        epochs=30, 
        save_dir="weights", 
        save_best=True, 
        save_last=True, 
        save_interval=5
    )
    
    # Load best model for predictions
    model = load_model(model, "weights/best_unetpp.pth", device)
    
    # Generate and save predictions
    save_predictions(model, train_loader, save_dir="predictions", device=device)
    save_predictions(model, val_loader, save_dir="predictions", device=device)
    
    print("Training and prediction complete!")

Using device: cuda
Found 799 valid image-mask pairs out of 799 images
Found 200 valid image-mask pairs out of 200 images

Epoch 1/30


100%|██████████| 50/50 [00:04<00:00, 11.04it/s]


Train Loss: 0.3161
Val Loss: 0.1822, Dice: 0.9476, IoU: 0.9028, Acc: 0.9707, Prec: 0.9421, Recall: 0.9544, F1: 0.9476
✅ Saved best model with Dice: 0.9476

Epoch 2/30


100%|██████████| 50/50 [00:04<00:00, 11.67it/s]


Train Loss: 0.1450
Val Loss: 0.1349, Dice: 0.9557, IoU: 0.9167, Acc: 0.9747, Prec: 0.9380, Recall: 0.9754, F1: 0.9557
✅ Saved best model with Dice: 0.9557

Epoch 3/30


...

Train Loss: 0.0809
Val Loss: 0.0820, Dice: 0.9695, IoU: 0.9414, Acc: 0.9827, Prec: 0.9688, Recall: 0.9708, F1: 0.9695

Epoch 30/30


100%|██████████| 50/50 [00:04<00:00, 11.57it/s]


Train Loss: 0.0645
Val Loss: 0.0706, Dice: 0.9730, IoU: 0.9479, Acc: 0.9848, Prec: 0.9743, Recall: 0.9723, F1: 0.9730
✅ Saved checkpoint at epoch 30
✅ Saved model after last epoch


C:\Users\Asus\AppData\Local\Temp\ipykernel_5844\1331862524.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location=device)


✅ Loaded model from epoch 25 with Dice score: 0.975209344625473
Training and prediction complete!
